# Utils

In [1]:
import numpy as np
import numpy.linalg as linalg
from typing import Tuple, List, Callable
from math import sqrt
import itertools
from enum import Enum
import pandas as pd
from itertools import product
import plotly.graph_objects as go
from timeit import default_timer as timer

In [2]:
def get_circle_radius(matrix: np.ndarray, row_index: int, squared: bool = False) -> float:
    if squared:
        return (matrix[row_index] ** 2).sum() - (matrix[row_index][row_index] ** 2)

    return abs(matrix[row_index]).sum() - abs(matrix[row_index][row_index])

In [3]:
def get_gershgorin_circles(matrix: np.ndarray) -> List[Tuple[float, float]]:
    circles = []
    for i in range(matrix.shape[0]):
        radius = get_circle_radius(matrix, i)
        center = matrix[i][i]
        circles.append((center - radius, center + radius))
    return circles

In [4]:
def rotate(matrix: np.ndarray, index: Tuple[int, int]) -> None:
    i, j = index

    matrix_ii, matrix_ij, matrix_jj = matrix[i][i], matrix[i][j], matrix[j][j]

    x = -2 * matrix_ij
    y = matrix_ii - matrix_jj

    if y == 0:
        cos_phi = sqrt(2) / 2
        sin_phi = sqrt(2) / 2
    else:
        cos_phi = sqrt(1 / 2 * (1 + abs(y) / sqrt(x ** 2 + y ** 2)))
        sin_phi = np.sign(x * y) * abs(x) / (2 * cos_phi * sqrt(x ** 2 + y ** 2))

    matrix[i][i] = cos_phi ** 2 * matrix_ii - 2 * sin_phi * cos_phi * matrix_ij + sin_phi ** 2 * matrix_jj
    matrix[j][j] = sin_phi ** 2 * matrix_ii + 2 * sin_phi * cos_phi * matrix_ij + cos_phi ** 2 * matrix_jj

    matrix[i][j] = (cos_phi ** 2 - sin_phi ** 2) * matrix_ij + sin_phi * cos_phi * (matrix_ii - matrix_jj)
    matrix[j][i] = (cos_phi ** 2 - sin_phi ** 2) * matrix_ij + sin_phi * cos_phi * (matrix_ii - matrix_jj)

    for k in range(matrix.shape[0]):
        if k != i and k != j:
            matrix_ik, matrix_jk = matrix[i][k], matrix[j][k]

            matrix[i][k] = cos_phi * matrix_ik - sin_phi * matrix_jk
            matrix[k][i] = cos_phi * matrix_ik - sin_phi * matrix_jk

            matrix[j][k] = sin_phi * matrix_ik + cos_phi * matrix_jk
            matrix[k][j] = sin_phi * matrix_ik + cos_phi * matrix_jk

In [5]:
def find_abs_max_index(matrix: np.ndarray) -> Tuple[int, int]:
    matrix = matrix - np.diag(np.diag(matrix))
    return np.unravel_index(np.argmax(abs(matrix)), matrix.shape)

In [6]:
def get_line_by_line_indices(shape: Tuple[int, int]):
    indicies = []
    for i in range(shape[0]):
        for j in range(shape[1]):
            if i != j:
                indicies.append((i, j))
    return itertools.cycle(indicies)

In [7]:
def get_vector_of_square_sums(matrix: np.ndarray) -> np.ndarray:
    return np.array([get_circle_radius(matrix, i, squared=True) for i in range(matrix.shape[0])])


def get_pivot_with_max_radius(matrix: np.ndarray, vector: np.ndarray) -> Tuple[int, int]:
    i = vector.argmax()
        
    matrix = matrix - np.diag(np.diag(matrix))
    j = abs(matrix[i]).argmax()

    return i, j

In [8]:
class PivotStrategy(Enum):
    ABS_MAX = 'abs_max'
    LINE_BY_LINE = 'line_by_line'
    MAX_RADIUS = 'max_radius'

In [9]:
def find_eigenvalues(
        matrix: np.ndarray,
        *,
        eps: float,
        strategy: PivotStrategy,
        limit: int = 10000,
) -> Tuple[np.ndarray, int]:
    matrix = np.copy(matrix)

    circle_indicies = get_line_by_line_indices(matrix.shape)
    square_sums_vector = get_vector_of_square_sums(matrix)

    for step in range(limit):
        if strategy is PivotStrategy.MAX_RADIUS:
            if all(get_circle_radius(matrix, i, squared=True) < eps ** 2 for i in range(matrix.shape[0])):
                return np.diag(matrix), step
        else:
            if all(get_circle_radius(matrix, i) < eps for i in range(matrix.shape[0])):
                return np.diag(matrix), step

        if strategy is PivotStrategy.MAX_RADIUS:
            index = get_pivot_with_max_radius(matrix, square_sums_vector)
            square_sums_vector[index[0]] = get_circle_radius(matrix, index[0], squared=True)
            square_sums_vector[index[1]] = get_circle_radius(matrix, index[1], squared=True)
            rotate(matrix, index)
        elif strategy is PivotStrategy.LINE_BY_LINE:
            index = next(circle_indicies)
            rotate(matrix, index)
        elif strategy is PivotStrategy.ABS_MAX:
            index = find_abs_max_index(matrix)
            rotate(matrix, index)

    return np.diag(matrix), step + 1

In [10]:
def benchmark_matrix(matrix: np.ndarray, eps: float) -> pd.Series:
    true_values, _ = linalg.eig(matrix)

    abs_max_start = timer()
    abs_max_values, abs_max_steps = find_eigenvalues(matrix, eps=eps, strategy=PivotStrategy.ABS_MAX)
    abs_max_finish = timer()

    line_by_line_start = timer()
    line_by_line_values, line_by_line_steps = find_eigenvalues(matrix, eps=eps, strategy=PivotStrategy.LINE_BY_LINE)
    line_by_line_finish = timer()

    radius_start = timer()
    radius_values, radius_steps = find_eigenvalues(matrix, eps=eps, strategy=PivotStrategy.MAX_RADIUS)
    radius_finish = timer()

    return pd.Series([
        abs_max_steps,
        abs_max_finish - abs_max_start,
        linalg.norm(abs(np.sort(true_values)) - abs(np.sort(abs_max_values))),
        line_by_line_steps,
        line_by_line_finish - line_by_line_start,
        linalg.norm(abs(np.sort(true_values)) - abs(np.sort(line_by_line_values))),
        radius_steps,
        radius_finish - radius_start,
        linalg.norm(abs(np.sort(true_values)) - abs(np.sort(radius_values))),
    ])

In [11]:
def compare_size_and_steps(data: pd.DataFrame, eps: float) -> None:
    data = data[data.eps == eps]

    fig = go.Figure()

    fig.add_scatter(x=data.n, y=data.abs_max_steps, name='Максимальный по модулю')
    fig.add_scatter(x=data.n, y=data.line_by_line_steps, name='По порядку')
    fig.add_scatter(x=data.n, y=data.radius_steps, name='Круг максимального радиуса')

    fig.update_xaxes(title='Размер матрицы')
    fig.update_yaxes(title='Количество шагов')

    fig.update_layout(title=f'Зависимость количества шагов от размера матрицы (ε = {eps})', legend_title_text='Стратегия')

    fig.show()

In [12]:
def compare_eps_and_steps(data: pd.DataFrame, n: int) -> None:
    data = data[data.n == n]

    fig = go.Figure()

    fig.add_scatter(x=data.eps, y=data.abs_max_steps, name='Максимальный по модулю')
    fig.add_scatter(x=data.eps, y=data.line_by_line_steps, name='По порядку')
    fig.add_scatter(x=data.eps, y=data.radius_steps, name='Круг максимального радиуса')

    fig.update_xaxes(title='ε', type='log', tickformat='.0e', autorange='reversed')
    fig.update_yaxes(title='Количество шагов')

    fig.update_layout(title=f'Зависимость количества шагов от ε ({n = })', legend_title_text='Стратегия')

    fig.show()

In [13]:
def compare_size_and_error(data: pd.DataFrame, eps: float) -> None:
    data = data[data.eps == eps]

    fig = go.Figure()

    fig.add_scatter(x=data.n, y=data.abs_max_error, name='Максимальный по модулю')
    fig.add_scatter(x=data.n, y=data.line_by_line_error, name='По порядку')
    fig.add_scatter(x=data.n, y=data.radius_error, name='Круг максимального радиуса')

    fig.update_xaxes(title='Размер матрицы')
    fig.update_yaxes(title='Ошибка', tickformat='.2e')

    fig.update_layout(title=f'Зависимость ошибки от размера матрицы (ε = {eps})', legend_title_text='Стратегия')

    fig.show()

In [14]:
def compare_eps_and_error(data: pd.DataFrame, n: int) -> None:
    data = data[data.n == n]

    fig = go.Figure()

    fig.add_scatter(x=data.eps, y=data.abs_max_error, name='Максимальный по модулю')
    fig.add_scatter(x=data.eps, y=data.line_by_line_error, name='По порядку')
    fig.add_scatter(x=data.eps, y=data.radius_error, name='Круг максимального радиуса')

    fig.update_xaxes(title='ε', type='log', tickformat='.0e', autorange='reversed')
    fig.update_yaxes(title='Ошибка', tickformat='.2e')

    fig.update_layout(title=f'Зависимость ошибки от ε ({n = })', legend_title_text='Стратегия')

    fig.show()

In [15]:
def compare_size_and_time(data: pd.DataFrame, eps: float) -> None:
    data = data[data.eps == eps]

    fig = go.Figure()

    fig.add_scatter(x=data.n, y=data.abs_max_time, name='Максимальный по модулю')
    fig.add_scatter(x=data.n, y=data.line_by_line_time, name='По порядку')
    fig.add_scatter(x=data.n, y=data.radius_time, name='Круг максимального радиуса')

    fig.update_xaxes(title='Размер матрицы')
    fig.update_yaxes(title='Время', tickformat='.2e')

    fig.update_layout(title=f'Зависимость времени работы от размера матрицы (ε = {eps})', legend_title_text='Стратегия')

    fig.show()

In [16]:
def compare_eps_and_time(data: pd.DataFrame, n: int) -> None:
    data = data[data.n == n]

    fig = go.Figure()

    fig.add_scatter(x=data.eps, y=data.abs_max_time, name='Максимальный по модулю')
    fig.add_scatter(x=data.eps, y=data.line_by_line_time, name='По порядку')
    fig.add_scatter(x=data.eps, y=data.radius_time, name='Круг максимального радиуса')

    fig.update_xaxes(title='ε', type='log', tickformat='.0e', autorange='reversed')
    fig.update_yaxes(title='Время работы', tickformat='.2e')

    fig.update_layout(title=f'Зависимость времени работы от ε ({n = })', legend_title_text='Стратегия')

    fig.show()

# Diagonal

In [17]:
def get_diagonal_matrix(size: int) -> np.ndarray:
    return np.diag(range(1, size + 1))

In [18]:
diagonal_data = pd.DataFrame(product(range(2, 101), [10 ** -i for i in range(2, 16)]), columns=['n', 'eps'])
diagonal_data[
    [
        'abs_max_steps', 
        'abs_max_time',
        'abs_max_error', 
        'line_by_line_steps', 
        'line_by_line_time',
        'line_by_line_error', 
        'radius_steps',
        'radius_time',
        'radius_error',
    ]
] = diagonal_data.apply(
    func=lambda row: benchmark_matrix(get_diagonal_matrix(int(row.n)), row.eps),
    axis=1,
)
diagonal_data

,n,eps,abs_max_steps,abs_max_time,abs_max_error,line_by_line_steps,line_by_line_time,line_by_line_error,radius_steps,radius_time,radius_error
0,2,1.000000e-02,0.0,0.000185,0.0,0.0,0.000070,0.0,0.0,0.000066,0.0
1,2,1.000000e-03,0.0,0.000092,0.0,0.0,0.000066,0.0,0.0,0.000063,0.0
2,2,1.000000e-04,0.0,0.000104,0.0,0.0,0.000067,0.0,0.0,0.000064,0.0
3,2,1.000000e-05,0.0,0.000087,0.0,0.0,0.000062,0.0,0.0,0.000061,0.0
4,2,1.000000e-06,0.0,0.000081,0.0,0.0,0.000062,0.0,0.0,0.000661,0.0
...,...,...,...,...,...,...,...,...,...,...,...
1381,100,1.000000e-11,0.0,0.003404,0.0,0.0,0.003193,0.0,0.0,0.015717,0.0
1382,100,1.000000e-12,0.0,0.003430,0.0,0.0,0.003248,0.0,0.0,0.007378,0.0
1383,100,1.000000e-13,0.0,0.003493,0.0,0.0,0.003271,0.0,0.0,0.021125,0.0
1384,100,1.000000e-14,0.0,0.003445,0.0,0.0,0.008615,0.0,0.0,0.003852,0.0


In [19]:
print(get_gershgorin_circles(get_diagonal_matrix(5)))
print(find_eigenvalues(get_diagonal_matrix(5), eps=1e-12, strategy=PivotStrategy.ABS_MAX)[0])
print(find_eigenvalues(get_diagonal_matrix(5), eps=1e-12, strategy=PivotStrategy.LINE_BY_LINE)[0])
print(find_eigenvalues(get_diagonal_matrix(5), eps=1e-12, strategy=PivotStrategy.MAX_RADIUS)[0])

[(1, 1), (2, 2), (3, 3), (4, 4), (5, 5)]
[1 2 3 4 5]
[1 2 3 4 5]
[1 2 3 4 5]


# Diagonally dominant

In [20]:
def get_diagonally_dominant_matrix(size: int) -> np.ndarray:
    return np.eye(size) * 4 + np.eye(size, k=1) * -1 + np.eye(size, k=-1) * -1

In [21]:
diagonally_dominant_data = pd.DataFrame(product(range(2, 31), [10 ** -i for i in range(2, 16)]), columns=['n', 'eps'])
diagonally_dominant_data[
    [
        'abs_max_steps',
        'abs_max_time',
        'abs_max_error', 
        'line_by_line_steps',
        'line_by_line_time',
        'line_by_line_error', 
        'radius_steps',
        'radius_time',
        'radius_error',
    ]
] = diagonally_dominant_data.apply(
    func=lambda row: benchmark_matrix(get_diagonally_dominant_matrix(int(row.n)), row.eps),
    axis=1,
)
diagonally_dominant_data

KeyboardInterrupt: KeyboardInterrupt: 

In [23]:
compare_size_and_steps(diagonally_dominant_data, eps=1e-2)
compare_size_and_steps(diagonally_dominant_data, eps=1e-8)
compare_size_and_steps(diagonally_dominant_data, eps=1e-15)

Unsupported

In [24]:
compare_eps_and_steps(diagonally_dominant_data, 7)
compare_eps_and_steps(diagonally_dominant_data, 25)

Unsupported

In [25]:
compare_size_and_time(diagonally_dominant_data, eps=1e-2)
compare_size_and_time(diagonally_dominant_data, eps=1e-8)
compare_size_and_time(diagonally_dominant_data, eps=1e-15)

Unsupported

In [26]:
compare_eps_and_time(diagonally_dominant_data, 7)
compare_eps_and_time(diagonally_dominant_data, 25)

Unsupported

In [27]:
compare_size_and_error(diagonally_dominant_data, 1e-2)
compare_size_and_error(diagonally_dominant_data, 1e-8)
compare_size_and_error(diagonally_dominant_data, 1e-15)

Unsupported

In [28]:
print(get_gershgorin_circles(get_diagonally_dominant_matrix(5)))
print(find_eigenvalues(get_diagonally_dominant_matrix(5), eps=1e-12, strategy=PivotStrategy.ABS_MAX)[0])
print(find_eigenvalues(get_diagonally_dominant_matrix(5), eps=1e-12, strategy=PivotStrategy.LINE_BY_LINE)[0])
print(find_eigenvalues(get_diagonally_dominant_matrix(5), eps=1e-12, strategy=PivotStrategy.MAX_RADIUS)[0])

[(3.0, 5.0), (2.0, 6.0), (2.0, 6.0), (2.0, 6.0), (3.0, 5.0)]
[5.         3.         5.73205081 2.26794919 4.        ]
[5.73205081 2.26794919 3.         5.         4.        ]
[3.         5.73205081 2.26794919 5.         4.        ]


# Hilbert

In [22]:
def generate_matrix(element_factory: Callable[[int, int], float], size: int) -> np.ndarray:
    return np.array(
        [
            [element_factory(row, column) for column in range(1, size + 1)]
            for row in range(1, size + 1)
        ],
        dtype=float,
    )


def get_hilbert_matrix(size: int) -> np.ndarray:
    return generate_matrix(lambda row, column: 1 / (row + column - 1), size)

In [23]:
hilbert_data = pd.DataFrame(product(range(2, 29), [10 ** -i for i in range(2, 16)]), columns=['n', 'eps'])
hilbert_data[
    [
        'abs_max_steps',
        'abs_max_time',
        'abs_max_error', 
        'line_by_line_steps',
        'line_by_line_time',
        'line_by_line_error', 
        'radius_steps',
        'radius_time',
        'radius_error',
    ]
] = hilbert_data.apply(
    func=lambda row: benchmark_matrix(get_hilbert_matrix(int(row.n)), row.eps),
    axis=1,
)
hilbert_data

,n,eps,abs_max_steps,abs_max_time,abs_max_error,line_by_line_steps,line_by_line_time,line_by_line_error,radius_steps,radius_time,radius_error
0,2,1.000000e-02,1.0,0.000201,2.237726e-16,1.0,0.000093,2.237726e-16,1.0,0.000133,2.237726e-16
1,2,1.000000e-03,1.0,0.000151,2.237726e-16,1.0,0.000089,2.237726e-16,1.0,0.000128,2.237726e-16
2,2,1.000000e-04,1.0,0.000188,2.237726e-16,1.0,0.000102,2.237726e-16,1.0,0.000159,2.237726e-16
3,2,1.000000e-05,1.0,0.000185,2.237726e-16,1.0,0.000102,2.237726e-16,1.0,0.000164,2.237726e-16
4,2,1.000000e-06,1.0,0.000196,2.237726e-16,1.0,0.000101,2.237726e-16,1.0,0.000165,2.237726e-16
...,...,...,...,...,...,...,...,...,...,...,...
373,28,1.000000e-11,895.0,0.141702,5.277773e-13,2413.0,0.248558,1.067210e-13,882.0,0.146152,1.135419e-11
374,28,1.000000e-12,994.0,0.156990,1.204290e-13,2580.0,0.273437,9.244751e-15,976.0,0.160946,4.782693e-13
375,28,1.000000e-13,1079.0,0.170472,2.713529e-14,2695.0,0.287614,8.407133e-15,1057.0,0.177724,2.120022e-14
376,28,1.000000e-14,1172.0,0.184557,2.897653e-15,3305.0,0.356298,2.053450e-15,1149.0,0.193077,2.649789e-15


In [31]:
compare_size_and_steps(hilbert_data, 1e-2)
compare_size_and_steps(hilbert_data, 1e-8)
compare_size_and_steps(hilbert_data, 1e-15)

Unsupported

In [32]:
compare_eps_and_steps(hilbert_data, 5)
compare_eps_and_steps(hilbert_data, 16)
compare_eps_and_steps(hilbert_data, 28)

Unsupported

In [33]:
compare_size_and_time(hilbert_data, 1e-2)
compare_size_and_time(hilbert_data, 1e-8)
compare_size_and_time(hilbert_data, 1e-15)

Unsupported

In [34]:
compare_eps_and_time(hilbert_data, 5)
compare_eps_and_time(hilbert_data, 16)
compare_eps_and_time(hilbert_data, 28)

Unsupported

In [36]:
print(get_gershgorin_circles(get_hilbert_matrix(4)))
print(find_eigenvalues(get_hilbert_matrix(4), eps=1e-12, strategy=PivotStrategy.ABS_MAX)[0])
print(find_eigenvalues(get_hilbert_matrix(4), eps=1e-12, strategy=PivotStrategy.LINE_BY_LINE)[0])
print(find_eigenvalues(get_hilbert_matrix(4), eps=1e-12, strategy=PivotStrategy.MAX_RADIUS)[0])

[(-0.08333333333333304, 2.083333333333333), (-0.6166666666666667, 1.2833333333333332), (-0.5499999999999998, 0.9499999999999997), (-0.47380952380952385, 0.7595238095238095)]
[1.50021428e+00 1.69141220e-01 9.67023040e-05 6.73827361e-03]
[1.50021428e+00 1.69141220e-01 9.67023040e-05 6.73827361e-03]
[1.50021428e+00 1.69141220e-01 9.67023040e-05 6.73827361e-03]
